In [1]:
%matplotlib inline


# Demo of DBSCAN clustering algorithm


Finds core samples of high density and expands clusters from them.




In [50]:
print(__doc__)

import numpy as np
from __future__ import absolute_import, division, print_function
import codecs
import csv
import glob
import re
import nltk
import os

import pandas as pd
from sklearn.cluster import KMeans
import codecs
import glob
import re
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.cm as cm
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


# # #############################################################################
# X = np.array([[0.309585,0.195831,-0.571898,0.320652,0.19859,-0.174276,-0.20575,-0.000975914,0.445632,-0.268902,-0.206201,-0.633367,-0.210247,-0.380637,0.759928,-0.782565,0.118909,-0.0114356,0.541104,-0.0486329,0.112736,-0.0450957,0.864808,-0.27075,0.0585939,-0.598973],
# [-0.44295,-0.138248,-0.304361,0.0841964,-0.461872,-0.0478393,-0.0417216,0.665065,0.863832,-0.363148,0.695037,0.207979,-0.0904296,-0.47487,0.678785,-0.278006,0.261033,0.136941,0.518635,0.762631,0.0376978,0.0785251,0.342366,-1.5589,1.20426,0.451576],
# [0.360239,-0.0517429,0.633756,-0.471496,0.037566,-0.05381,0.0279278,0.660736,-0.284218,-0.0735615,-0.0390197,0.547344,0.131321,-0.191338,-0.0230034,-0.633198,-0.138865,0.241388,0.210924,-0.120586,0.177218,-0.271775,-0.849061,-0.303331,-0.28994,0.495727],
# [0.0117626,0.537253,0.309631,-0.0755103,0.153332,0.163334,-0.439986,-0.0341552,0.266899,-0.0604446,0.496762,-0.170136,-0.596482,-0.334571,-0.263573,-0.378634,0.375341,0.323421,0.487746,0.133938,-0.301779,-0.286481,0.00875104,0.69636,-0.0436106,1.06787],
# [-0.25611,-0.0727603,0.431058,-0.549209,-1.30984,1.57982,0.447918,-0.730619,-0.378204,-2.12272,1.24083,-0.965105,0.300642,-0.261183,1.50461,0.271094,1.05007,-0.0575936,-0.843349,-0.991438,0.169101,-2.1807,-0.166426,-0.360071,0.899028,-0.375755],
# [-0.983737,-1.2389,0.114062,1.07773,0.639809,0.600226,0.126243,0.255459,1.38529,-0.0208233,1.28195,-0.610213,1.63623,-1.13292,-0.427356,0.237173,-0.175981,-0.865972,-2.1652,-1.07257,-0.175388,0.123563,0.636433,-0.27866,-0.221435,-0.0191743],
# [0.471158,0.36723,-1.41692,0.30476,-0.706457,-1.2747,-0.766756,0.51255,-1.2624,-0.214485,1.31904,-1.66939,1.2217,-0.813697,1.23599,0.376103,2.12699,-0.628152,-0.341696,0.794546,-1.59937,-1.57728,-0.940015,1.30501,-0.678183,0.00979086],
# [0.477313,0.45283,-0.0533268,-0.356297,0.384119,-0.569675,0.486929,-0.364685,-0.233344,0.457348,0.122171,-0.156767,0.378387,-0.296002,0.302162,-0.476906,-0.509059,-0.590632,0.508541,-0.221997,-0.298075,0.30205,-0.0969545,0.0762017,-0.0995773,0.483804],
# [-0.606732,-0.576231,0.245052,-0.175648,-0.346522,-0.624487,0.312291,0.231276,-0.293516,-0.859501,-0.475011,-0.0185644,-0.242426,0.5073,1.26082,0.412908,0.587412,-0.0204924,-0.275689,-0.1345,-0.667593,0.107205,0.44611,0.176265,0.0742499,-1.12428],
# [-0.217914,-0.141174,0.56235,-0.284602,-0.0372247,1.30065,-1.66497,0.151772,0.118544,-1.45761,0.275819,-0.272581,1.19147,-0.709617,-0.221312,0.951102,-0.347294,0.473456,-0.686838,-0.699319,-0.700191,0.382785,-0.976863,0.144829,-0.444214,0.354953],
# [-0.00335689,-0.0328241,0.223185,-0.498965,0.573359,0.229933,0.291253,-0.338691,0.674246,-0.403187,0.492287,0.444174,-0.637693,-0.45504,0.24364,0.0535675,-0.49591,-0.147483,0.182188,-0.234961,-0.427906,0.00708592,0.548628,-0.139199,-0.184445,0.203779],
# [-0.696233,-0.920563,-0.135777,-0.33791,0.289537,-0.394755,0.290693,0.742433,0.246248,2.09288,-1.30178,-0.0667954,-0.357638,0.945725,-0.0614894,1.00628,0.138342,0.275441,-0.482669,0.221103,-1.3037,0.593788,0.538469,0.870838,0.401921,-0.436914],
# [-1.38449,-1.02277,0.372173,-0.461975,-0.0368364,0.176027,0.241481,1.70447,0.536987,-1.59459,1.25025,0.562844,0.385462,0.216856,-0.161597,-0.631466,0.420019,-0.454913,-0.83376,-1.11178,-0.36905,0.689395,-0.718804,0.655449,-1.1272,0.355262],
# [-0.258601,-0.0374118,-0.157491,-0.11191,0.103313,0.0836609,0.21948,-0.116656,-0.456605,0.074102,0.135214,-0.019602,0.112263,0.0853853,0.481193,-0.330206,0.0956202,0.0827785,0.287751,0.228265,-0.144567,0.0851071,-0.385265,0.195806,-0.041831,0.0140121],
# [-0.107213,0.0601869,-0.39103,-0.317413,0.275206,-0.0818625,-0.20737,-0.562109,-0.576702,0.145652,0.0651688,0.745743,1.43884,-0.114053,-0.193478,-0.0588002,0.329102,-0.430371,0.606933,-0.557867,0.158338,-0.0795058,-0.0153516,-0.751533,0.178585,0.296034],
# [-1.62374,0.225437,-1.32588,-0.967665,-0.287894,-1.21134,0.647011,-0.551025,0.55287,-0.41171,-1.84728,-0.731413,0.0755348,-1.33578,-0.292421,-0.851923,1.39215,-0.707892,-1.12687,0.276333,-0.906669,-0.489017,1.07167,-0.32619,-0.49361,1.08357],
# [-0.148253,0.0538499,-0.285887,0.0630424,0.678505,0.127975,-0.515998,0.799492,-0.294782,0.0865924,-0.411248,-0.35809,-0.857694,-0.385853,0.541388,-0.51699,0.616896,0.0334215,0.0788914,-0.935854,0.224073,-0.243898,0.573263,0.267552,-0.289148,0.449478],
# [1.33344,-0.554274,-0.886441,0.152149,-0.866785,1.0165,-0.472063,-0.311098,0.648155,-0.569091,-0.96221,0.195363,1.76939,0.0887395,0.246691,-0.743982,1.4339,0.469539,0.344724,-1.40923,-1.91407,-0.616154,0.117883,0.0441184,0.763991,-0.376414],
# [-0.016874,0.173005,1.01546,0.217416,1.26386,-0.0730285,-1.55299,-0.237292,0.0972881,2.17956,-0.359201,1.44756,1.01724,-0.370202,-1.65321,-0.277063,0.547188,-1.39763,-0.54324,0.787001,-0.876272,0.683741,-0.207837,-0.759445,-1.40484,-0.106977],
# [1.49481,0.517189,-2.03671,0.0620483,-0.260722,-0.236512,0.808269,0.490972,-0.701941,-0.0348551,-0.88181,-0.265216,-1.25623,0.157786,0.857345,0.709908,1.2439,-0.874592,-0.336329,0.68854,1.12663,0.634589,-0.493748,0.545846,0.748441,-0.791566],
# [0.673782,-0.0407811,0.463654,0.277617,0.143893,0.115255,0.00802584,0.707729,-0.135528,0.756134,0.0196977,-0.500748,-0.410792,-0.759493,0.563104,-0.309588,0.528621,0.62347,0.0781774,-0.318702,-0.202916,0.0854159,-0.154125,-0.396808,-0.597821,0.179846],
# [0.149148,0.173139,0.31001,-0.00840967,0.132381,0.0660034,0.0191679,-0.199699,0.312287,-0.106559,0.297927,0.23792,-0.278408,-0.409361,-0.569865,-0.0169484,-0.0145474,-0.307813,0.0866914,0.282878,-0.106147,-0.38943,0.237188,-0.140337,-0.415692,0.135971],
# [-0.175362,0.320607,-0.239719,0.67643,0.509637,0.18087,1.35807,0.115728,-0.235288,0.419705,0.602464,0.361033,0.204954,-0.225943,0.3065,0.174914,0.783998,-0.343113,-0.384643,-0.111226,-0.431854,0.487537,-0.039413,0.417968,-0.697856,-0.109138],
# [-0.274139,-1.08659,-0.359829,-0.952721,0.150627,-0.0140903,0.370812,-0.373393,0.0328169,-0.255042,-0.148522,-0.374651,-0.262389,0.628279,0.249913,0.150493,0.268785,0.280373,-0.0937156,1.09121,-0.0275999,0.0983629,-0.153128,0.327578,0.295962,-0.381438],
# [-0.253705,-0.0366784,0.515859,-0.0853706,0.00268233,-0.699484,-0.300198,-0.201446,0.769876,-0.253979,-0.591371,-0.660558,0.0106869,-0.131671,0.106594,-0.625727,0.0297691,0.0529777,0.167315,-0.286704,-0.0619753,0.00071381,-0.00915578,-0.501495,0.259648,0.136341]])
vectors = pd.read_csv("data/Archive (1)/test-body-vectors.csv" )
X = vectors[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200']]
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=1, min_samples=1, metric = 'euclidean').fit(X)
print("done")
# # core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# # core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# # # Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("n-clusters: ",n_clusters_)
print(labels)
# # print('Estimated number of clusters: %d' % n_clusters_)
# # print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# # print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# # print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# # print("Adjusted Rand Index: %0.3f"
# #       % metrics.adjusted_rand_score(labels_true, labels))
# # print("Adjusted Mutual Information: %0.3f"
# #       % metrics.adjusted_mutual_info_score(labels_true, labels))
# # print("Silhouette Coefficient: %0.3f"
# #       % metrics.silhouette_score(X, labels))

# # Plot result
# import matplotlib.pyplot as plt

# # Black removed and is used for noise instead.
# unique_labels = set(labels)
# colors = [plt.cm.Spectral(each)
#           for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]

#     class_member_mask = (labels == k)

#     xy = X[class_member_mask & core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=14)

#     xy = X[class_member_mask & ~core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=6)

# plt.title('Estimated number of clusters: %d' % n_clusters_)
# plt.show()

Automatically created module for IPython interactive environment
done
n-clusters:  16227
[    0     1     2 ..., 16224 16225 16226]
